In [32]:
import csv
import os
import numpy as np

def pergunta2(i):
    file = "part0" + "dev" + str(i+1) + ".csv"
    pasta = "part0"
    dir = os.path.join(pasta, file)
    dev = []
    with open(dir) as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in reader:
            linha = [float(n) for n in row]
            dev.append(linha)

    return np.array(dev)

for i in range(5):
    dev1 = pergunta2(i)
    dev2 = pergunta2(i)
    dev3 = pergunta2(i)
    dev4 = pergunta2(i)
    dev5 = pergunta2(i)
    

